In [2]:
#!pip install lightfm

In [118]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix, coo_matrix

# Матричная факторизация
from implicit.als import AlternatingLeastSquares
from implicit.nearest_neighbours import bm25_weight, tfidf_weight

from lightfm import LightFM
from lightfm.evaluation import precision_at_k, recall_at_k

# Функции из 1-ого вебинара
import os, sys

module_path = os.path.abspath(os.path.join(os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)
    
#from src.metrics import precision_at_k, recall_at_k
#from src.utils import prefilter_items

In [1]:
def prefilter_items(data, take_n_popular=5000, item_features=None):
    # Уберем самые популярные товары (их и так купят)
    popularity = data.groupby('item_id')['user_id'].nunique().reset_index() / data['user_id'].nunique()
    popularity.rename(columns={'user_id': 'share_unique_users'}, inplace=True)

    top_popular = popularity[popularity['share_unique_users'] > 0.5].item_id.tolist()
    data = data[~data['item_id'].isin(top_popular)]

    # Уберем самые НЕ популярные товары (их и так НЕ купят)
    top_notpopular = popularity[popularity['share_unique_users'] < 0.01].item_id.tolist()
    data = data[~data['item_id'].isin(top_notpopular)]

    popularity = data.groupby('item_id')['quantity'].sum().reset_index()
    popularity.rename(columns={'quantity': 'n_sold'}, inplace=True)

    top_5000 = popularity.sort_values('n_sold', ascending=False).head(take_n_popular).item_id.tolist()
    data.loc[~data['item_id'].isin(top_5000), 'item_id'] = 999999

    # Уберем товары, которые не продавались за последние 12 месяцев
#     items_sold_last365 = data.loc[(data['day'] > data['day'].max() - 365) & (data['day'] != 0)]['item_id'].unique()
#     data = data.loc[data['item_id'].isin(items_sold_last365) == True]

    # Уберем не интересные для рекомендаций категории (department)
    # этот фильтр пока нереализован, поскольку нет точных указаний, какие категории считать не интересными

    # Уберем слишком дешевые товары (на них не заработаем). 1 покупка из рассылок стоит 60 руб.
    # Уберем все товары, где количество купленных = 0 и где сумма покупки, деленная на количество менее 60
    # (таких товаров осталось мало!!! следует провести анализ единиц, в которых указана сумма покупки)
#     data = data.loc[data['quantity'] != 0]
#     data = data.loc[(data['sales_value'] / data['quantity']) > 60]

    # Уберем слишком дорогие товары
    # этот фильтр пока нереализован. Необходимо уточнение, какие товары считаем слишком дорогими

    return data


In [120]:
data = pd.read_csv('retail_train.csv')
item_features = pd.read_csv('product.csv')
user_features = pd.read_csv('hh_demographic.csv')

# column processing
item_features.columns = [col.lower() for col in item_features.columns]
user_features.columns = [col.lower() for col in user_features.columns]

item_features.rename(columns={'product_id': 'item_id'}, inplace=True)
user_features.rename(columns={'household_key': 'user_id'}, inplace=True)

# train test split
test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

data_train.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [143]:
data_train_filtered = prefilter_items(data_train, take_n_popular=5000, item_features=item_features)
data_test_filtered = prefilter_items(data_test, take_n_popular=5000, item_features=item_features)

In [122]:
# 2.1. Подготовка csr-обучающей матрицы: 

user_item_matrix = pd.pivot_table(data_train_filtered, 
                                  index='user_id', columns='item_id', 
                                  values='quantity', # Можно пробоват ьдругие варианты
                                  aggfunc='count', 
                                  fill_value=0
                                 )

user_item_matrix = user_item_matrix.astype(float) # необходимый тип матрицы для implicit

# переведем в формат saprse matrix
sparse_user_item = csr_matrix(user_item_matrix).tocsr()

user_item_matrix.head(2)

item_id,202291,397896,420647,480014,545926,707683,731106,818980,819063,819227,...,15778533,15831255,15926712,15926775,15926844,15926886,15927403,15927661,15927850,16809471
user_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [123]:
# 2.2. Подготовка csr-тестовой матрицы: 

data_test = data_test[data_test['item_id'].isin(data_train['item_id'].unique())]

test_user_item_matrix = pd.pivot_table(data_test_filtered, 
                                  index='user_id', columns='item_id', 
                                  values='quantity', # Можно пробовать другие варианты
                                  aggfunc='count', 
                                  fill_value=0
                                 )

test_user_item_matrix = test_user_item_matrix.astype(float) # необходимый тип матрицы для implicit
test_user_item_matrix.head(2)

item_id,397896,480014,707683,819063,819112,819304,819308,819330,819518,819765,...,17215026,17249007,17249614,17290664,17290929,17328876,17330026,17827215,17827241,17974316
user_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [124]:
userids = user_item_matrix.index.values
itemids = user_item_matrix.columns.values

matrix_userids = np.arange(len(userids))
matrix_itemids = np.arange(len(itemids))

id_to_itemid = dict(zip(matrix_itemids, itemids))
id_to_userid = dict(zip(matrix_userids, userids))

itemid_to_id = dict(zip(itemids, matrix_itemids))
userid_to_id = dict(zip(userids, matrix_userids))

In [125]:
# 3. Подготовка user и item признаков: 

user_feat = pd.DataFrame(user_item_matrix.index)
user_feat = user_feat.merge(user_features, on='user_id', how='left')
user_feat.set_index('user_id', inplace=True)
user_feat.head(2)

,age_desc,marital_status_code,income_desc,homeowner_desc,hh_comp_desc,household_size_desc,kid_category_desc
user_id,,,,,,,
1,65+,A,35-49K,Homeowner,2 Adults No Kids,2,None/Unknown
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [126]:
item_feat = pd.DataFrame(user_item_matrix.columns)
item_feat = item_feat.merge(item_features, on='item_id', how='left')
item_feat.set_index('item_id', inplace=True)

item_feat.head(2)

,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product
item_id,,,,,,
202291,69.0,MISC SALES TRAN,Private,COUPON/MISC ITEMS,GASOLINE-REG UNLEADED,
397896,69.0,KIOSK-GAS,Private,COUPON/MISC ITEMS,GASOLINE-REG UNLEADED,


In [127]:
# 4. Кодирование признаков: 

user_feat_lightfm = pd.get_dummies(user_feat, columns=user_feat.columns.tolist())
item_feat_lightfm = pd.get_dummies(item_feat, columns=item_feat.columns.tolist())

Подбор гиперпараметров: 

In [130]:
loss_list = ['bpr', 'warp']

metrics_dict = {}
for loss in loss_list:

    model = LightFM(no_components=20,
                    loss=loss,
                    learning_rate=0.01, 
                    item_alpha=0.4,
                    user_alpha=0.1, 
                    random_state=42,
                    k=5,
                    n=15,
                    max_sampled=100)
    
    model.fit((sparse_user_item > 0) * 1,  # user-item matrix из 0 и 1
              sample_weight=coo_matrix(user_item_matrix),
              user_features=csr_matrix(user_feat_lightfm.values).tocsr(),
              item_features=csr_matrix(item_feat_lightfm.values).tocsr(),
              epochs=15, 
              num_threads=4,
              verbose=False) 
    
    test_precision = precision_at_k(model, csr_matrix(test_user_item_matrix).tocsr(), 
                                 user_features=csr_matrix(user_feat_lightfm.values).tocsr(),
                                 item_features=csr_matrix(item_feat_lightfm.values).tocsr(),
                                 k=5).mean()
    
    metrics_dict[loss] = test_precision
print(metrics_dict)

{'bpr': 0.0042115576, 'warp': 0.0070519103}


In [131]:
%%time

components_list = [10, 20, 50]

metrics_dict = {}
for comp in components_list:

    model = LightFM(no_components=comp,
                    loss='warp',
                    learning_rate=0.01, 
                    item_alpha=0.4,
                    user_alpha=0.1, 
                    random_state=42,
                    k=5,
                    n=15,
                    max_sampled=100)
    
    model.fit((sparse_user_item > 0) * 1,  # user-item matrix из 0 и 1
              sample_weight=coo_matrix(user_item_matrix),
              user_features=csr_matrix(user_feat_lightfm.values).tocsr(),
              item_features=csr_matrix(item_feat_lightfm.values).tocsr(),
              epochs=15, 
              num_threads=4,
              verbose=False) 
    
    test_precision = precision_at_k(model, csr_matrix(test_user_item_matrix).tocsr(), 
                                 user_features=csr_matrix(user_feat_lightfm.values).tocsr(),
                                 item_features=csr_matrix(item_feat_lightfm.values).tocsr(),
                                 k=5).mean()
    
    metrics_dict[no_comp] = test_precision
print(metrics_dict)

{10: 0.0055827624, 20: 0.0070519103, 50: 0.004505387}
CPU times: user 7min, sys: 39.6 ms, total: 7min
Wall time: 1min 48s


In [134]:
%%time

rate_list = [0.001, 0.01, 0.05]

metrics_dict = {}
for rate in rate_list:

    model = LightFM(no_components=20,
                    loss='warp',
                    learning_rate=rate, 
                    item_alpha=0.4,
                    user_alpha=0.1, 
                    random_state=42,
                    k=5,
                    n=15,
                    max_sampled=100)
    
    model.fit((sparse_user_item > 0) * 1,  # user-item matrix из 0 и 1
              sample_weight=coo_matrix(user_item_matrix),
              user_features=csr_matrix(user_feat_lightfm.values).tocsr(),
              item_features=csr_matrix(item_feat_lightfm.values).tocsr(),
              epochs=15, 
              num_threads=4,
              verbose=False) 
    
    test_precision = precision_at_k(model, csr_matrix(test_user_item_matrix).tocsr(), 
                                 user_features=csr_matrix(user_feat_lightfm.values).tocsr(),
                                 item_features=csr_matrix(item_feat_lightfm.values).tocsr(),
                                 k=5).mean()
    
    metrics_dict[rate] = test_precision
print(metrics_dict)

{0.001: 0.0044074436, 0.01: 0.0070519103, 0.05: 0.0050930465}
CPU times: user 5min 34s, sys: 71.9 ms, total: 5min 34s
Wall time: 1min 26s


In [135]:
%%time

user_alpha_list = [0.01, 0.05, 0.1]

metrics_dict = {}
for user_alpha in user_alpha_list:

    model = LightFM(no_components=20,
                    loss='warp',
                    learning_rate=0.01, 
                    item_alpha=0.1,
                    user_alpha=user_alpha, 
                    random_state=42,
                    k=5,
                    n=15,
                    max_sampled=100)
    
    model.fit((sparse_user_item > 0) * 1,  # user-item matrix из 0 и 1
              sample_weight=coo_matrix(user_item_matrix),
              user_features=csr_matrix(user_feat_lightfm.values).tocsr(),
              item_features=csr_matrix(item_feat_lightfm.values).tocsr(),
              epochs=15, 
              num_threads=4,
              verbose=False) 
    
    test_precision = precision_at_k(model, csr_matrix(test_user_item_matrix).tocsr(), 
                                 user_features=csr_matrix(user_feat_lightfm.values).tocsr(),
                                 item_features=csr_matrix(item_feat_lightfm.values).tocsr(),
                                 k=5).mean()
    
    metrics_dict[user_alpha] = test_precision
print(metrics_dict)

{0.01: 0.0061704214, 0.05: 0.0070519103, 0.1: 0.004505387}
CPU times: user 5min 59s, sys: 79.9 ms, total: 5min 59s
Wall time: 1min 32s


In [142]:
%%time

max_sampled_list = [10, 30, 50, 100, 500]

metrics_dict = {}
for max_s in max_sampled_list:

    model = LightFM(no_components=20,
                    loss='warp',
                    learning_rate=0.01, 
                    item_alpha=0.1,
                    user_alpha=0.05, 
                    random_state=42,
                    k=5,
                    n=15,
                    max_sampled=max_s)
    
    model.fit((sparse_user_item > 0) * 1,  # user-item matrix из 0 и 1
              sample_weight=coo_matrix(user_item_matrix),
              user_features=csr_matrix(user_feat_lightfm.values).tocsr(),
              item_features=csr_matrix(item_feat_lightfm.values).tocsr(),
              epochs=15, 
              num_threads=4,
              verbose=False) 
    
    test_precision = precision_at_k(model, csr_matrix(test_user_item_matrix).tocsr(), 
                                 user_features=csr_matrix(user_feat_lightfm.values).tocsr(),
                                 item_features=csr_matrix(item_feat_lightfm.values).tocsr(),
                                 k=5).mean()
    
    metrics_dict[max_s] = test_precision
print(metrics_dict)

{10: 0.0053868755, 30: 0.0070519103, 50: 0.0067580803, 100: 0.0070519103, 500: 0.0070519103}
CPU times: user 9min 43s, sys: 71.9 ms, total: 9min 43s
Wall time: 2min 30s


Лучшая модель с параметрами:

    model = LightFM(no_components=20,
                    loss='warp',
                    learning_rate=0.01, 
                    item_alpha=0.1,
                    user_alpha=0.05, 
                    random_state=42,
                    k=5,
                    n=15,
                    max_sampled=30)
с метрикой, равной 0.007

In [144]:
train_precision = precision_at_k(model, sparse_user_item, 
                                 user_features=csr_matrix(user_feat_lightfm.values).tocsr(),
                                 item_features=csr_matrix(item_feat_lightfm.values).tocsr(),
                                 k=5).mean()

train_precision

0.5592637

In [145]:
model

In [146]:
test_precision = precision_at_k(model, csr_matrix(test_user_item_matrix).tocsr(), 
                                 user_features=csr_matrix(user_feat_lightfm.values).tocsr(),
                                 item_features=csr_matrix(item_feat_lightfm.values).tocsr(),
                                 k=5).mean()
test_precision

0.0070519103

По итогам анализа train и test precision есть предположение, что модель переобучается, необходимо провести дополнительный анализ тренировочной и тестовой выбооки и регуляризации